In [ ]:
import json


In [ ]:
# Load the risk rating mapping and decision tree data
def load_data(filepath):
    """Loads data from a JSON file."""
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

risk_mapping = load_data('risk_rating_mapping.json')
decision_tree = load_data('credit_rating_decision_tree_v2.json')

In [ ]:
# Define a function to traverse the decision tree and assign a credit rating
def assess_creditworthiness(entity, tree):
    """Recursively assesses creditworthiness based on the decision tree."""
    node = tree['tree']

    def traverse(node, entity):
        if node['type'] == 'root':
            for child in node['children']:
                return traverse(child, entity)
        elif node['type'] == 'decision':
            question = node['question']
            if question == 'Is the borrower a company or a sovereign entity?':
                if entity['Type'] == 'Company':
                    for child in node['children']:
                        if child['condition'] == 'Company':
                            next_node_id = child['node_id']
                            next_node = next((n for n in tree['tree']['children'] if n['node_id'] == next_node_id), None)
                            return traverse(next_node, entity)
                elif entity['Type'] == 'Sovereign':
                    for child in node['children']:
                        if child['condition'] == 'Sovereign':
                            next_node_id = child['node_id']
                            next_node = next((n for n in tree['tree']['children'] if n['node_id'] == next_node_id), None)
                            return traverse(next_node, entity)
        elif node['type'] == 'factor':
            # For simplicity, we'll assume a score is directly assigned here.
            # In a real system, we'd need to evaluate the metrics within the factor.
            total_score = 0
            for child in node['children']:
              for grand_child in child['children']:
                total_score += grand_child['score'] # Simplified scoring
            return total_score * node['weight']
        elif node['type'] == 'leaf':
            return node['score']
        return None

    final_score = traverse(node, entity)
    return final_score

In [ ]:
# Map the score to a credit rating
def map_score_to_rating(score, rating_scale):
    """Maps a creditworthiness score to an S&P credit rating."""
    for range_str, rating in rating_scale.items():
        lower, upper = map(int, range_str.split('-'))
        if lower <= score <= upper:
            return rating
    return 'D'  # Default rating if score is out of range

In [ ]:
# Example Usage
entities = [
    {'Name': 'Company A', 'Type': 'Company', 'Return on Equity': 20, 'Operating Margin': 12, 'Net Income Growth': 8, 'Debt-to-Equity Ratio': 1.0, 'Debt-to-Asset Ratio': 0.4, 'Operating Cash Flow': 150000, 'Free Cash Flow': 70000, 'Current Ratio': 2.5, 'Quick Ratio': 1.2, 'Industry': 'Technology', 'Growth Prospects': 'High', 'Competitive Landscape': 'Favorable', 'Quality of Leadership': 'Strong', 'Strategic Direction': 'Clear', 'Cost Structure': 'Efficient', 'Asset Turnover': 'High', 'Macroeconomic Conditions': 'Good', 'Regulatory Environment': 'Favorable', 'ESG Factors': 'Positive'},
    {'Name': 'Country B', 'Type': 'Sovereign', 'Real GDP Growth': 4, 'GDP per Capita': 25000, 'Economic Diversification': 'High', 'Government Debt-to-GDP Ratio': 50, 'Budget Balance': 'Surplus', 'Current Account Balance': 'Sustainable', 'Foreign Currency Reserves': 'Adequate', 'Political Stability': 'High', 'Rule of Law': 'Strong', 'Corruption Control': 'Effective', 'Geopolitical Risk': 'Low', 'Social Cohesion': 'High', 'Income Inequality': 'Low', 'Debt Structure': 'Favorable', 'Debt Service Capacity': 'Strong', 'Access to International Capital Markets': 'Good', 'External Financing Needs': 'Manageable', 'Contingent Liabilities': 'Low'}
]

rating_scale = risk_mapping['rating_scale']

for entity in entities:
    score = assess_creditworthiness(entity, decision_tree)
    rating = map_score_to_rating(score, rating_scale)
    print(f"{entity['Name']} Credit Rating: {rating} (Score: {score:.2f})" )